Todo:
- audio dataset automatic donload with code from kaggle
- make web scraping for text more flexible with names so it won't throw out an error whenever name doesnt match exactly
- do the eda with this rough data to see if it has everything we need for the pipline.
- create new file with buidling pipline so we won't have to rerun eda everytime.

# 1. Imports

## 1.1 Packages

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
import librosa
import random
import time
import requests
from urllib.parse import unquote

## 1.2 Audio Dataset From kaggle.com

In [3]:
audio_path = "data/audio/original_dataset/"
audio_files = os.listdir(audio_path)
audio_df = pd.DataFrame({
    'model': [os.path.splitext(f)[0] for f in audio_files],
    'path': [os.path.join(audio_path, f) for f in audio_files]
})
display(audio_df.head())

,model,path
0,38sws-dot38-caliber,data/audio/original_dataset/38sws-dot38-caliber
1,ak-12,data/audio/original_dataset/ak-12
2,ak-47,data/audio/original_dataset/ak-47
3,arctic-warfare-magnum,data/audio/original_dataset/arctic-warfare-magnum
4,beretta-92,data/audio/original_dataset/beretta-92


## 1.3 Image API From wikimedia.org

In [3]:
from image_api import ImageAPI
API_URL = "https://commons.wikimedia.org/w/api.php"
HEADERS = {"User-Agent": "Pipline/1.0 (mkakol.index@gmail.com)"}
audio_files = os.listdir(audio_path)
models = [os.path.splitext(f)[0] for f in audio_files]
df = ImageAPI(API_URL, HEADERS).fetch_images_for_models(models)

Searching for 38sws-dot38-caliber...
No image found for 38sws-dot38-caliber.
Searching for ak-12...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/c/c2/5%2C45mm_AK-12_6P70_assault_rifle_at_Military-technical_forum_ARMY-2016_03_noBG.png
Searching for ak-47...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/f/f6/AK-47_assault_rifle.jpg
Searching for arctic-warfare-magnum...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/a/a6/Accuracy_International_AWM_Users.png
Searching for beretta-92...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/2/24/Beretta_92S_%28cropped%29.jpg
Searching for colt-1911...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/c/cf/Colt_M1911_Pistol_from_1912_Army_manual.png
Searching for daewoo-k2...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/5/59/Daewoo_K2_rifle_1.jpg
Searching for desert-eagle...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/8/81/2014-_Desert_Eagle_.50_Action_Express

## 1.4 Web Scraping Text From gunsfandom.com

In [4]:
from text_webscraping import get_fandom_text
url = "https://guns.fandom.com/wiki/AK-47"
paras = get_fandom_text(url)
for i, p in enumerate(paras[:10]):
    print(p)

<p class="sitenotice-wrapper__title">Gun Wiki</p>
<p><font face="Verdana">
</font></p>
<p><font face="Verdana"><b><span style="font-size:160%; color:#FF0000;">M.O.S.</span> —</b> <span style="font-size:120%; color:#000;">Please read and keep the <b><a href="/wiki/Gun_Wiki:Manual_of_Style" title="Gun Wiki:Manual of Style">Manual of Style</a></b> in mind while editing.</span>
</font></p>
<p><font face="Verdana"><b><span style="font-size:120%; color:#FF0000;">Also, remember that there is to be no copying and pasting from other sources, at any time - your edits must be in your OWN words!</span></b>
</font></p>
<p>READ MORE</p>
<p>
<aside class="portable-infobox pi-background pi-border-color pi-theme-wikia pi-layout-default" role="region">
<div class="pi-image-collection wds-tabber" data-source="image">
<div class="wds-tabs__wrapper">
<ul class="wds-tabs">
<li class="wds-tabs__tab wds-is-current">
<span class="wds-tabs__tab-label">
							AK-47
							
						</span>
</li>
<li class="wds-tab

In [5]:
# from text_webscraping import get_fandom_text
# url = "https://guns.fandom.com/wiki/colt-1911"
# paras = get_fandom_text(url)
# for i, p in enumerate(paras[:10]):
#    print(p)

# 2. EDA

## 2.1 Audio EDA

We nemen audio dataset als base, omdat die als enige komt meteen als structured database die kunnen wij als fundatie voor images en text gebruiken. Elke subfolder is genaamd met een firearm, dat is ook onze identifier die gaan wij gebruiken om data te vinden via API en web scraping. Dit is handing, omdat dan hoeven wij de namen niet hand mating invullen en dus is de beste manier om te doen voor onze pipline.

Het audio dataset heeft totaal 1480 bestanden die zijn evenveel spread out tussen 58 unique firearmen die hebben wij gedownload. Dit betekendt dat voor elke unique firearm er zijn in totaal 25 verschillende audio bestanden die kunnen wij gebruiken om betere inzicht in data te krijgen. Het doel is om op basis van dat een gemiddelde te trekken van "avg_shots_fired_sec" feature om te kijken of er correlatie bestaat tussen dat en target variable "weapon_type" die kan gelableld worden als "pistol", "rifle" of "shotgun".

Elke sampel is al 0.5 seconden lang, met normalized HZ rate naar 44100 HZ, en dus alle filen zijn ook hetzelefde grotte (43KB). Het is ook verwacht, omdat dit is een structured dataset en dus de kwaliteit en preprocessing van de data is al op best wel hoge nivou. Wat moet nog gedaan worden is metadata van de audios te trekken om feature "avg_shots_per_sec"

Peaks


niet geldig
Wat wisten al dat audios waaren noramlized en dus het "loudeness" nivou is niet meer mogelijk om te extracteren om te kijken hoe druk eenf irearm zou zijn.

In [1]:
from audio_dataset import audio_eda
audio_df, counts, dur, srates, sizes = audio_eda("data/audio/original_dataset")

Audio files: 1450
Unique firearms: 58
Audios per firearm: {'38sws-dot38-caliber': 25, 'ak-12': 25, 'ak-47': 25, 'arctic-warfare-magnum': 25, 'beretta-92': 25, 'colt-1911': 25, 'daewoo-k2': 25, 'desert-eagle': 25, 'double-barrel-shotgun': 25, 'dp-27-lmg': 25, 'fn-p90': 25, 'fn-scar': 25, 'glock': 25, 'glock-18c': 25, 'glock-19-9mm-luger-pistol': 25, 'high-standard-22lr': 25, 'hk-g36c': 25, 'hk-ump45': 25, 'hk-usp-compact-40-sw-pistol': 25, 'kar98k': 25, 'kimber-45acp': 25, 'kriss-vector': 25, 'lorcin-380acp': 25, 'm16': 25, 'm24-sws': 25, 'm249-lmg': 25, 'm4': 25, 'mg42-lmg': 25, 'mini-14': 25, 'mk14-ebr': 25, 'mp-40-40-sw-pistol': 25, 'mp5-smg': 25, 'nagant-m1895': 25, 'ots-14-groza': 25, 'pp-19-bizon': 25, 'pump-action-shotgun': 25, 'qbu-88': 25, 'qbz-95': 25, 'remington-22lr': 25, 'remington-700': 25, 'remington-870': 25, 'rhino-60ds': 25, 'ruger-22lr': 25, 'ruger-357': 25, 'ruger-ar-556': 25, 'saiga-12k': 25, 'sig-p225': 25, 'sks-rifle': 25, 'slr-rifle': 25, 'steyr-aug': 25, 'sw-10-

## 2.2 Image EDA

## 2.3 Text EDA

In [6]:
df.head()

,model,url,path
0,ak-12,https://upload.wikimedia.org/wikipedia/commons...,data/images\ak-12.jpg
1,ak-47,https://upload.wikimedia.org/wikipedia/commons...,data/images\ak-47.jpg
2,arctic-warfare-magnum,https://upload.wikimedia.org/wikipedia/commons...,data/images\arctic-warfare-magnum.jpg
3,beretta-92,https://upload.wikimedia.org/wikipedia/commons...,data/images\beretta-92.jpg
4,colt-1911,https://upload.wikimedia.org/wikipedia/commons...,data/images\colt-1911.jpg


# 3 Preprocessing

## 3.1 Preprocessing

In [6]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy.signal import find_peaks

def audio_feature_extraction(audio_root, hop_length=512, peak_prominence=0.1, min_shot_interval_s=0.05):
    rows = []
    for firearm in os.listdir(audio_root):
        fdir = os.path.join(audio_root, firearm)
        if not os.path.isdir(fdir):
            continue
        for fname in os.listdir(fdir):
            path = os.path.join(fdir, fname)
            y, sr = librosa.load(path, sr=None, mono=True)
            duration = len(y) / sr

            # RMS envelope
            frame_len = int(0.01 * sr)
            rms = librosa.feature.rms(y=y, frame_length=frame_len, hop_length=hop_length)[0]
            times = librosa.frames_to_time(np.arange(len(rms)), sr=sr, hop_length=hop_length, n_fft=frame_len)

            # Peak detection
            prominence = peak_prominence * np.max(rms)
            peaks, _ = find_peaks(rms, prominence=prominence, distance=int(min_shot_interval_s * sr / hop_length))

            # Filter peaks by min interval
            peak_times = times[peaks]
            if len(peak_times) > 1:
                cleaned = [peak_times[0]]
                for t in peak_times[1:]:
                    if t - cleaned[-1] >= min_shot_interval_s:
                        cleaned.append(t)
                peak_times = np.array(cleaned)

            shots = len(peak_times)
            shots_per_s = shots / duration if duration > 0 else np.nan

            rows.append({
                "firearm": firearm,
                "filename": fname,
                "shots": shots,
                "shots_per_s": shots_per_s,
                "peak_db": 20 * np.log10(np.max(np.abs(y))+1e-12),
            })

    df = pd.DataFrame(rows)
    return df

def aggregate_firearm_features(df):
    agg = df.groupby("firearm").agg(
        avg_shots_per_sec=("shots_per_s","mean"),
        avg_peak_db=("peak_db","mean")
    ).reset_index()
    return agg

# Example usage
file_df = audio_feature_extraction("data/audio/original_dataset")
per_firearm = aggregate_firearm_features(file_df)

print(per_firearm)

                        firearm  avg_shots_per_sec  avg_peak_db
0           38sws-dot38-caliber               5.52   -11.508130
1                         ak-12               6.64    -1.709352
2                         ak-47               6.64    -1.177706
3         arctic-warfare-magnum               6.96    -1.281113
4                    beretta-92               6.08    -1.835327
5                     colt-1911               6.00    -0.748771
6                     daewoo-k2               7.92    -0.880832
7                  desert-eagle               8.00    -3.758838
8         double-barrel-shotgun               7.36    -2.073422
9                     dp-27-lmg               9.04    -1.633819
10                       fn-p90               9.44    -6.889853
11                      fn-scar               9.84    -1.145473
12                        glock               6.00   -14.327656
13                    glock-18c               7.12    -1.090604
14    glock-19-9mm-luger-pistol         

## 3.2 Image Preprocessing

## 3.3 Text Preprocessing